MACRO
==================

In [ ]:
options ls=max;  * so the SAS log lines do not wrap ;
options mprint;  * print macro generated code ;

* create source data ;
data source (index=(acct));
  length acct parent $3 amount 8;
  input acct parent amount;
  datalines;
A1  .   10
A2  A1  20
A3  A2  30
A4  A2  20
A5  A1  10
A6  A5  10
A7  .   10
A8  A7  20
A9  A7  30
A10 A9  20
;
run;

%macro code;
* option 1: index key lookup to derive top-level parent ;

* use recursion to get the top level parent ;
/* Note: This is my first version...I prefer the second approach
data option1;
  set source;
  temp=acct;
  do while (1);
    if missing(parent) then do;
      parent=acct;
      acct=temp;
      leave;
    end;
    acct=parent;
    set source (drop=amount) key=acct / unique;
  end;
  _error_=0;
  drop temp;
run;
*/

data option1;
  set source;
  _acct=acct;
  _parent=parent;
  do while (not missing(parent));
    acct=parent;
    set source (drop=amount) key=acct / unique;
  end;
  toplevel=acct;
  acct=_acct;
  parent=_parent;
  _error_=0;
  drop _:;
run;

proc summary data=option1 nway noprint;
  class toplevel;
  var amount;
  output out=sum1 sum=;
run;